In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import label_binarize

In [2]:
df = pd.read_csv('../data/aden_golfe.csv',sep=';')

In [3]:
max_quinzaine=24
max_area_lat=12
max_area_lon=31
number_grid=(max_area_lat+1)*(max_area_lon+1)
print df.head()

   Annee (0)  jour (0)  mois (0)  dist �� la c��te (nq) zone lat (0)  long(0)  \
0       2010        28        10                    NaN  BCD  12:08N  054:25E   
1       2010        30        10                    NaN  BCD  15:06N  055:58E   
2       2010         1         1                    NaN  BCD  12:31N  047:18E   
3       2010         3         2                    NaN  BCD  13:04N  047:04E   
4       2010         5         2                    NaN  BCD  13:00N  048:45E   

           type(0)           pavillon statut de l'attaque(0)      ...       \
0             Dhow         Not Stated               Steaming      ...        
1     Bulk Carrier              Malta               Steaming      ...        
2  Chemical Tanker          Singapore               Steaming      ...        
3    General Cargo        North Korea               Steaming      ...        
4     Bulk Carrier  Antigua & Barbuda               Steaming      ...        

    latitude  longitude  quinzaine_week  \
0

In [4]:
def monthCorrect(month):
    month=str(int(month))
    if len(month)==1:
        return '0'+month
    else:
        return month

def fortnight(day):
    if int(day) <15:
        return 1
    else:
        return 2

def activity(years):
    if int(years) <2013:
        return 2
    else:
        return 1
    

In [5]:
df["mois (0)"]=df.apply(lambda row: monthCorrect(row['mois (0)']), axis=1)
df["jour (0)"]=df.apply(lambda row: monthCorrect(row['jour (0)']), axis=1)
df["date"]=df.apply(lambda row: str(row["Annee (0)"]) +"-"+ str(row["mois (0)"]), axis=1)
df["fortnight"]=df.apply(lambda row: fortnight(row["jour (0)"]), axis=1)
df["activity"]=df.apply(lambda row: fortnight(row["Annee (0)"]), axis=1)

# Version 1.1 : group by years, month, area

In [35]:
df['lat_area'] = df['lat_area'].astype(int)
df['lon_area'] = df['lon_area'].astype(int)
df['area'] = df.apply(lambda row: str(row['lat_area'])+'_'+str(row['lon_area']), axis=1)

## Creation d'un data frame vide

In [36]:
dfAgregateTotal=pd.DataFrame({'count' : df.groupby( [ "Annee (0)","mois (0)","area"] ).size()}).reset_index()

In [37]:
specifecArea=pd.DataFrame(dfAgregateTotal['area'].str.split('_',1).tolist(),columns = ['lat_area','lon_area'])
dfAgregateTotal=pd.concat([dfAgregateTotal, specifecArea], axis=1)
print dfAgregateTotal.head(20)
print len(dfAgregateTotal)

dfAgregateTotal['mois (0)']=dfAgregateTotal['mois (0)'].astype(int).astype(str)
dfAgregateTotal['lat_area']=dfAgregateTotal['lat_area'].astype(int)
dfAgregateTotal['lon_area']=dfAgregateTotal['lon_area'].astype(int)

    Annee (0) mois (0)  area  count lat_area lon_area
0        2008       02  2_12      2        2       12
1        2008       03  1_13      1        1       13
2        2008       03  2_11      1        2       11
3        2008       03  3_11      1        3       11
4        2008       04  2_10      1        2       10
5        2008       04  2_11      1        2       11
6        2008       04   2_9      1        2        9
7        2008       04  3_10      2        3       10
8        2008       04   3_9      1        3        9
9        2008       05  2_11      2        2       11
10       2008       05   2_8      1        2        8
11       2008       05   2_9      2        2        9
12       2008       05  3_11      1        3       11
13       2008       06  2_10      1        2       10
14       2008       06   2_8      1        2        8
15       2008       07  2_10      1        2       10
16       2008       07  2_11      1        2       11
17       2008       07   2_3

In [38]:
uniqueLat=dfAgregateTotal[["lat_area","lon_area"]].values
unique=pd.Series(zip(uniqueLat[:,0], uniqueLat[:,1])).unique()


print "--------------------------------------------"
list1, list2 = zip(*unique)
array1=np.asarray(list1)
array2=np.asarray(list2)

ind = np.lexsort((array1,array2)) 
uniqueSort=[(array1[i],array2[i]) for i in ind]



print "--------------------------------------------"

list1, list2 = zip(*uniqueSort)
array1_=np.asarray(list1)
array2_=np.asarray(list2)
a = np.column_stack((array1_,array2_))
#print a


--------------------------------------------
--------------------------------------------


In [39]:
uniqueYears=dfAgregateTotal['Annee (0)'].unique()
print "-------------------------------------------------------------------"
ListAreaLat=np.tile(a[:,0],12*len(uniqueYears))
ListAreaLon=np.tile(a[:,1],12*len(uniqueYears))

uniqueMonth=np.linspace(1, 12, num=12)
ColMonth=np.repeat(uniqueMonth,len(a[:,1]))
#print ColMonth[1:100]
ColMonth=np.tile(ColMonth,len(uniqueYears))


Colyears=np.repeat(uniqueYears, 12*len(a[:,1]))
count=np.repeat(0, len(Colyears))


print len(ColMonth)
print len(Colyears)
print len(ListAreaLat)
print len(ListAreaLon)

-------------------------------------------------------------------
3864
3864
3864
3864


## Fill the matrice

In [40]:
#columns = ['Years','2_week', 'lat_area','lon_area','count']
#matriceML=pd.DataFrame(columns=columns)
#matriceML = matriceML.fillna(0)

dic = {'Years': Colyears, 
     'Month': ColMonth, 
     'lat_area': ListAreaLat,
     'lon_area': ListAreaLon,
     'count': count
      }

dfFinal= pd.DataFrame(dic)

dfFinal["Month"]=dfFinal.apply(lambda row: monthCorrect(row['Month']), axis=1)
dfFinal['Month']=dfFinal['Month'].astype(int)
dfFinal['lat_area']=dfFinal['lat_area'].astype(int)
dfFinal['lon_area']=dfFinal['lon_area'].astype(int)

print dfFinal.head(10)
print dfFinal.dtypes
print dfFinal[(dfFinal['Years']==2008) &
                (dfFinal['lat_area']==4) &
                (dfFinal['lon_area']==20 )&
                (dfFinal['Month']==1)]

   Month  Years  count  lat_area  lon_area
0      1   2008      0         4         1
1      1   2008      0         5         1
2      1   2008      0         2         2
3      1   2008      0         3         2
4      1   2008      0         4         2
5      1   2008      0         1         3
6      1   2008      0         2         3
7      1   2008      0         3         3
8      1   2008      0         1         4
9      1   2008      0         2         4
Month       int64
Years       int64
count       int64
lat_area    int64
lon_area    int64
dtype: object
Empty DataFrame
Columns: [Month, Years, count, lat_area, lon_area]
Index: []


In [41]:
for index, row in dfAgregateTotal.iterrows():
    #print "---------------------------------------------------------"
    #print "len : "+str(len(dfFinal['Years']==row['Annee (0)']))
    
    #print row['Annee (0)'], int(row['mois (0)']), row['count'] ,  row['lat_area'] , row['lon_area'] 
    
    
    idx=dfFinal[(dfFinal['Years']==row['Annee (0)']) &
                (dfFinal['lat_area']==row['lat_area']) &
                (dfFinal['lon_area']==row['lon_area'] )&
                (dfFinal['Month']==int(row["mois (0)"]))]
    
    #print len(idx)
    #print 'index : '+str(idx.index.values)
    #print 'count : '+str(row['count'])
    #dfFinal.set_value(idx.index.values, 'count',row['count'])
    dfFinal.set_value(idx.index.values, "count",row['count'])

In [42]:
print dfFinal['count'].sum()
print len(dfFinal)
print dfFinal[dfFinal['count']!=0].head()

421
3864
     Month  Years  count  lat_area  lon_area
73       2   2008      2         2        12
116      3   2008      1         2        11
117      3   2008      1         3        11
122      3   2008      1         1        13
157      4   2008      1         2         9


## Write matrice in a file

In [43]:
dfFinal.to_csv("../data/aden_golfe_matrix_area.csv",sep=';', encoding='utf-8', index=False)

#  

# Version 1.1 : group by years, month, area

In [6]:
df.head()

,Annee (0),jour (0),mois (0),dist �� la c��te (nq),zone,lat (0),long(0),type(0),pavillon,statut de l'attaque(0),...,newDescriptif,Guns,mousson,distance au port,new_type,large_area,area_by_hand,date,fortnight,activity
0,2010,28,10,NaN,BCD,12:08N,054:25E,Dhow,Not Stated,Steaming,...,pirates hijacked a dhow.,0,0,534.679104,Other,G,M,2010-10,2,2
1,2010,30,10,NaN,BCD,15:06N,055:58E,Bulk Carrier,Malta,Steaming,...,armed pirates attacked and boarded the ship un...,0,0,292.330113,Carrier,G,L,2010-10,2,2
2,2010,01,01,NaN,BCD,12:31N,047:18E,Chemical Tanker,Singapore,Steaming,...,pirates hijacked a tanker underway and took he...,0,0,259.217741,Tanker,G,I,2010-01,1,2
3,2010,03,02,NaN,BCD,13:04N,047:04E,General Cargo,North Korea,Steaming,...,armed pirates attacked and hijacked a ship and...,0,0,233.133566,Cargo,G,I,2010-02,1,2
4,2010,05,02,NaN,BCD,13:00N,048:45E,Bulk Carrier,Antigua & Barbuda,Steaming,...,six pirates in a high-speed boat armed with ma...,1,0,414.693842,Carrier,G,I,2010-02,1,2


In [9]:
print df["area_by_hand"].unique()

['M' 'L' 'I' 'G' 'J' 'C' 'D' 'F' 'K' 'E' 'A' 'B' nan]


In [7]:
dfAgregateByArea=pd.DataFrame({'count' : df.groupby( [ "activity","mois (0)","area_by_hand","fortnight","new_type"] ).size()}).reset_index()

In [8]:
dfAgregateByArea.head(10)
print dfAgregateByArea["count"].sum()

420


### Creation empty dataframe

In [56]:
uniqueArea=sorted(dfAgregateByArea["area_by_hand"].unique())
#uniqueYears=dfAgregateByArea['Annee (0)'].unique()
uniqueTypeBoat=dfAgregateByArea['new_type'].unique()
uniqueMonth=np.linspace(1, 12, num=12)
uniqueBinary=np.linspace(1, 2, num=2)

ColArea=np.tile(uniqueArea,len(uniqueMonth)*len(uniqueTypeBoat)*4)
ColMonth=np.repeat(uniqueMonth, len(uniqueArea)*4*len(uniqueTypeBoat))
ColType=np.repeat(np.tile(uniqueTypeBoat, len(uniqueArea)),4*len(uniqueMonth))
ColActivity=np.tile(np.repeat(uniqueBinary, 2*len(uniqueArea)),len(uniqueMonth)*len(uniqueTypeBoat))
ColFortnight=np.tile(np.repeat(uniqueBinary, len(uniqueArea)),2*len(uniqueMonth)*len(uniqueTypeBoat))
ColCount=np.repeat(0, len(ColFortnight))

print ColArea[0:100]
print ColMonth[0:100]
print ColType[0:100]
print ColActivity[0:100]

print len(ColArea)
print len(ColMonth)
print len(ColType)
print len(ColActivity)
print len(ColFortnight)

#print len(ColCount)

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'I' 'J' 'K' 'L' 'M' 'A' 'B' 'C' 'D' 'E' 'F'
 'G' 'I' 'J' 'K' 'L' 'M' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'I' 'J' 'K' 'L' 'M'
 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'I' 'J' 'K' 'L' 'M' 'A' 'B' 'C' 'D' 'E' 'F'
 'G' 'I' 'J' 'K' 'L' 'M' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'I' 'J' 'K' 'L' 'M'
 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'I' 'J' 'K' 'L' 'M' 'A' 'B' 'C' 'D' 'E' 'F'
 'G' 'I' 'J' 'K' 'L' 'M' 'A' 'B' 'C' 'D']
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
['Carrier' 'Carrier' 'Carrier' 'Carrier' 'Carrier' 'Carrier' 'Carrier'
 'Carrier' 'Carrier' 'Carrier' 'Carrier' 'Carrier' 'Carrier' 'Carrier'
 'Carrier' 'Carrier' 'Carrier' 'Carrier' 'Ca

### Fill dataframe

In [63]:
dic = {'Month': ColMonth,'Activity': ColActivity,'Fortnight': ColFortnight, 'Area': ColArea,'Type':ColType,'Count': ColCount}
dfFinalArea= pd.DataFrame(dic)

for index, row in dfAgregateByArea.iterrows():
    #print "---------------------------------------------------------"
    #print row['Annee (0)'], int(row['mois (0)']), row['count'] ,  row['area_by_hand']   
    
    idx=dfFinalArea[(dfFinalArea['Activity']==row['activity']) & (dfFinalArea['Area']==row['area_by_hand']) 
                    & (dfFinalArea['Type']==row['new_type'])& (dfFinalArea['Fortnight']==row['fortnight'])
                    & (dfFinalArea['Month']==int(row["mois (0)"]))]
    #print len(idx)
    dfFinalArea.set_value(idx.index.values, "Count",row['count'])

print dfFinalArea.head(40)
print dfFinalArea["Count"].sum()

    Activity Area  Count  Fortnight  Month     Type
0          1    A      0          1      1  Carrier
1          1    B      0          1      1  Carrier
2          1    C      0          1      1  Carrier
3          1    D      0          1      1  Carrier
4          1    E      0          1      1  Carrier
5          1    F      0          1      1  Carrier
6          1    G      0          1      1  Carrier
7          1    I      0          1      1  Carrier
8          1    J      0          1      1  Carrier
9          1    K      0          1      1  Carrier
10         1    L      0          1      1  Carrier
11         1    M      0          1      1  Carrier
12         1    A      0          2      1  Carrier
13         1    B      0          2      1  Carrier
14         1    C      0          2      1  Carrier
15         1    D      0          2      1  Carrier
16         1    E      0          2      1  Carrier
17         1    F      0          2      1  Carrier
18         1

In [64]:
dfFinalArea.to_csv("../data/aden_golfe_matrix_area_hand.csv",sep=';', encoding='utf-8', index=False)

# Version 2 : group by Date, type attack

In [77]:
dfAgregateTotalType=pd.DataFrame({'count' : df.groupby( [ "date","new_type"] ).size()}).reset_index()
dfAgregateTotalType["date"]=dfAgregateTotalType["date"]+"-01"
print dfAgregateTotalType.head(10)

         date new_type  count
0  2008-02-01   Tanker      1
1  2008-02-01      Tug      1
2  2008-03-01    Cargo      1
3  2008-03-01   Tanker      1
4  2008-03-01   Vessel      1
5  2008-04-01    Cargo      1
6  2008-04-01  Carrier      1
7  2008-04-01   Tanker      2
8  2008-04-01   Vessel      2
9  2008-05-01    Cargo      2


In [80]:
max_month=12
unique_years=sorted(df["Annee (0)"].unique())
unique_type=sorted(dfAgregateTotalType["new_type"].unique())
month_repeat=np.linspace(1, max_month, num=max_month)

colYears=np.repeat(unique_years, max_month)
colMonth=np.tile(month_repeat, len(unique_years))
                               
dic2 = {'Years': colYears, 'Month': colMonth}
dfGlobale = pd.DataFrame(dic2)

for i in range(0,len(unique_type)):
    nameColLat=str(unique_type[i])
    dfGlobale[nameColLat]=0


dfGlobale.head()
dfGlobale["Month"]=dfGlobale.apply(lambda row: monthCorrect(row['Month']), axis=1)
dfGlobale["date"]=dfGlobale.apply(lambda row: str(row["Years"]) +"-"+ str(row["Month"])+"-01", axis=1)
dfGlobale = dfGlobale.drop('Month', 1)                            
dfGlobale = dfGlobale.drop('Years', 1)
print dfGlobale.head()

   Barge  Cargo  Carrier  Other  Ro-Ro  Tanker  Tug  Vessel        date
0      0      0        0      0      0       0    0       0  2008-01-01
1      0      0        0      0      0       0    0       0  2008-02-01
2      0      0        0      0      0       0    0       0  2008-03-01
3      0      0        0      0      0       0    0       0  2008-04-01
4      0      0        0      0      0       0    0       0  2008-05-01


# Fill matrice

In [81]:
for index, row in dfAgregateTotalType.iterrows():
    #print 'Years '+str(row['Annee (0)']),'2_week : '+str( row['quinzaine_week']),'count : '+str(row['count'])#,'type_attack : '+str(row['type d\'attaque(0)'])

    idx=dfGlobale[(dfGlobale['date']==row['date'])]
    dfGlobale.set_value(idx.index.values, str(row["new_type"]),row['count'])

print dfGlobale.head(100)

    Barge  Cargo  Carrier  Other  Ro-Ro  Tanker  Tug  Vessel        date
0       0      0        0      0      0       0    0       0  2008-01-01
1       0      0        0      0      0       1    1       0  2008-02-01
2       0      1        0      0      0       1    0       1  2008-03-01
3       0      1        1      0      0       2    0       2  2008-04-01
4       0      2        2      1      0       1    0       0  2008-05-01
5       0      1        0      1      0       0    0       0  2008-06-01
6       0      0        2      1      0       1    0       0  2008-07-01
7       0      3        3      0      0       3    0       0  2008-08-01
8       0      3        6      3      0       7    0       0  2008-09-01
9       0      2        5      2      0       7    0       0  2008-10-01
10      0      1        5      1      0       8    0       1  2008-11-01
11      1      3        3      0      0       2    1       0  2008-12-01
12      0      4        5      2      1       7    

In [82]:
dfGlobale.to_csv("../data/aden_golfe_matrix_global.csv",sep=';', encoding='utf-8', index=False)